# The Machine Learning Workflow

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

### Load Data

#### Define target and features

#### Train-Test Split

## Feature Engineering

## Modelling

## Hyperparameter Optimization

## Cross validation